In [1]:
import pandas as pd
import numpy as np
import ast
from tqdm import tqdm_notebook as tqdm

In [3]:
from nltk import WhitespaceTokenizer

In [4]:
data = pd.read_csv('data/rap_corpus.csv', sep=';')
data.shape

(5420, 2)

In [5]:
data.head()

,title,text
0,Многоточие (DOTSFAM) - Интро DOTS FAMILY,"['Dots Family!', 'Fuck you!', 'Dots Family!', ..."
1,Паук - Мой бит,"['Припев (4 раза):', 'Мой бит качает.', 'Он ни..."
2,Стриж - Алена,"['Ухоженная девочка', 'Брильянтовые стразы', '..."
3,Скато (Skato) - Хроника,"['Кровоточат раны, швы наложены на прошлое.', ..."
4,Bad Balance - Африка,['- Как уже сообщалось: в минувшую субботу поз...


In [6]:
data.tail()

,title,text
5415,"Dessar - Постой, не уходи","['Я тушу боль последнею сигаретой ', 'В глазах..."
5416,Big Black Boots - Нам хорошо (feat. Тэона),"['Вот и мы, а где были вы', 'Когда мы начинали..."
5417,Каста - Гончая,"['На спидометре сто двадцать- мчим,', 'Трек по..."
5418,Krec - В никуда,"['Взять чемодан и умчать в никуда, ', 'Может, ..."
5419,Schokk - Грустная песня (feat. Maxat),"['Я иногда, будто сквозь время вижу в степи ру..."


In [7]:
## преобразование текста в список
rapers = []
texts = []
for i in tqdm(range(data.shape[0])):
    tmp = data.iloc[i,:]
    rapers.append(tmp.title.split(' - ')[0])
    txt = ast.literal_eval(tmp.text)
    txt = [x.lower() for x in txt]
    texts.append(txt)

In [8]:
print(len(rapers))
print(len(texts))

5420
5420


In [9]:
def string_processing(string):
    symbols = ['%', '&', '(', ')', '*', '+', '/', '[', '\\', ']', '`',
               '{', '}', '~', '«', '»','’','“','”','•','…','№','\ufeff', '\xa0', '=', '^']
    for s in symbols:
        string = string.replace(s, '')
    
    string = string.replace('$', 's')
    string = string.replace('ü', 'у')
    string = string.replace('є', 'е')
    string = string.replace('і', 'i')
    string = string.replace('ї', 'i')
    string = string.replace('ң', 'н')
    string = string.replace('à', 'а')
    string = string.replace('ß', 'b')
    string = string.replace('ó', 'о')
    string = string.replace('ù', 'и')
    
    string = string.replace('–', '-')
    string = string.replace('—', '-')
    string = string.replace('_', '-')
    string = string.replace('@', ' ')
    
    
    
    string = string.replace('.', ' ')
    string = string.replace('?', '')
    string = string.replace('!', '')
    string = string.replace(',', '')
    string = string.replace(' - ', ' ')
    string = string.replace(' -', ' ')
    string = string.replace('- ', '')
    string = string.replace('"', '')

    string = string.replace(':', '')
    string = string.replace(';', '')
    string = string.replace('   ', ' ')
    string = string.replace('  ', ' ')
    
    
    string = string.strip()
    
    if string.endswith(','):
        string = string[:-1]
        
    if string.endswith('-'):
        string = string[:-1]        
    
    return string

In [10]:
## чистка
texts_ = []
rapers_ = []
i = 0
for txt in tqdm(texts):
    raper = rapers[i]
    i += 1
    # проверка на наличие маркеров белых строк
    if txt.count('<w>') == 0:
        continue
    
    tmp = []    
    for string in txt:
        
        string = string_processing(string)
        
        if string == '':
            continue
        if string == ' ':
            continue
        if string == '  ':
            continue
        if string.startswith('[') and string.endswith(']'):
            continue
            
        if len(string) <= 5 and string != '<w>': #########
            continue
#         if re.match('^[0-9][.: ]', string):
#             continue            
        
        if string.count('припев') > 0:
            continue
        if string.count('куплет') > 0:
            continue
    
        tmp.append(string)

    texts_.append(tmp)
    rapers_.append(raper)


print(len(texts_))
print(len(rapers_))                


5100
5100


In [11]:
texts_[0][:5]

['мой бит качает',
 'он никого не напрягает',
 '<w>',
 'мой микрофон включён мой ди-джей на месте',
 'приготовьтесь к уроку и все вместе']

In [12]:
for txt in tqdm(texts_):
    txt.append('<w>')

In [13]:
## определение ритмической структуры песни
structures = []


for txt in tqdm(texts_):
    
    struct = []
    for i in range(len(txt)):
        if txt[i] == '<w>': struct.append(i)
    structures.append(struct)        

In [14]:
structures[:10]

[[2, 19, 22, 39, 42, 59, 62],
 [32, 37, 42, 63, 68, 73],
 [15, 20, 37, 42, 59, 64],
 [1, 18, 23, 32, 42, 59, 73, 77],
 [33, 38, 43, 69, 74, 79, 132, 134, 139, 144],
 [9, 36, 46, 65, 75, 85],
 [19, 24, 41, 46, 61, 66],
 [1, 18, 35, 52, 70, 72, 90, 110],
 [2, 17, 20, 37, 44, 47],
 [20, 25, 42, 47, 64, 69, 70, 75]]

In [15]:
texts_[7][:5]

['а напоследок я бухну ха-ха-ха',
 '<w>',
 'вечереет',
 'солнце скидывает яркую одежду',
 'на безбрежном небосводе облака шептаться стали нежно']

In [16]:
rapers_[-1]

'Schokk'

In [17]:
print(len(structures))
print(len(texts_))

5100
5100


In [25]:
strof = []
author = []

for struct in tqdm(structures):
    index = structures.index(struct)
    raper = rapers_[index]
    txt = texts_[index]
    
    start = 0
    for next_ in struct:
        strof_count = next_ - start - 1
#         # двустишия
#         if strof_count == 2:
#             s = []
#             s.append(txt[start+1])
#             s.append(txt[start+2])
#             s.extend(s)
#             if not s in strof:
#                 strof.append(s)
#                 author.append(raper)
#             start = next_
            
        # четверостишия
        if (strof_count) in [4, 8, 16, 32] and strof_count > 0:
            for i in range(start, start + strof_count, 4): # цикл по четверостишиям
                s = []
                for j in range(i+1, i+5, 1): # цикл по строкам в четверостишии
                    s.append(txt[j])
                if not s in strof and s[0] != s[1] and s[0] != s[2] and s[0] != s[3] :
                    # print(strof_count, s) 
                    strof.append(s)
                    author.append(raper)
            start = next_
        else:
            start = next_
            continue        
    
    
    
print(len(strof))
print(len(author))                


16413
16413


In [26]:
strof[:100]

[['мой микрофон включён мой ди-джей на месте',
  'приготовьтесь к уроку и все вместе',
  'поднимите руки вверх когда я толкаю тему',
  'теорему о том как атаковать по биту смело'],
 ['мои рифмы оставляют шрамы на сердцах',
  'наводят страх я вижу ты в моих руках',
  'ведь я всегда говорю правду даже когда вру',
  'я врождённый теоретик и буду им пока дышу'],
 ['пока не умру не обрету покой на том свете',
  'словно дети в моём уме бушует буйный ветер',
  'мои темы бесконечный экспресс злой пресс',
  'для всех фальшивых мс вечный стресс'],
 ['в моих строках виден блеск как в волнах плеск эмоций',
  'ненависть к врагам как революция в камбодже',
  'мои люди не хотят просто так спокойно жить',
  'быть как пугливое стадо а как солдаты всё бомбить'],
 ['жизнь как продажное дерьмо суть ничто',
  'мне плевать на всё я буду жить как суждено',
  'я верю только богу мою дорогу',
  'освещает звезда что выше разума дороже слова'],
 ['сил очень мало но моя вера велика',
  'она как река залита кровью

In [78]:
pairs0 = []
pairs1 = []
for s in tqdm(strof):
    s0 = WhitespaceTokenizer().tokenize(s[0])[-1]
    s1 = WhitespaceTokenizer().tokenize(s[1])[-1]
    s2 = WhitespaceTokenizer().tokenize(s[2])[-1]
    s3 = WhitespaceTokenizer().tokenize(s[3])[-1]
    
    if len(s0) < 2 or len(s1) < 2 or len(s2) < 2 or len(s3) < 2:
        continue
        
    if s0.isdigit() or s1.isdigit() or s2.isdigit() or s3.isdigit():
        continue

    if s0 != s1:
        pairs0.append(s0)
        pairs1.append(s1)

    if s0 != s2:
        pairs0.append(s0)
        pairs1.append(s2)
        
    if s0 != s3:
        pairs0.append(s0)
        pairs1.append(s3)    

    if s1 != s2:
        pairs0.append(s1)
        pairs1.append(s2)
        
    if s1 != s3:
        pairs0.append(s1)
        pairs1.append(s3)
    
    if s2 != s3:
        pairs0.append(s2)
        pairs1.append(s3)
    
    

In [79]:
for p1, p2 in zip(pairs0[:100], pairs1[:100]):
    print(p1, p2)

месте вместе
месте тему
месте смело
вместе тему
вместе смело
тему смело
сердцах руках
сердцах вру
сердцах дышу
руках вру
руках дышу
вру дышу
свете ветер
свете пресс
свете стресс
ветер пресс
ветер стресс
пресс стресс
эмоций камбодже
эмоций жить
эмоций бомбить
камбодже жить
камбодже бомбить
жить бомбить
ничто суждено
ничто дорогу
ничто слова
суждено дорогу
суждено слова
дорогу слова
велика иногда
велика тайну
велика байки
иногда тайну
иногда байки
тайну байки
истории море
истории идей
истории музей
море идей
море музей
идей музей
оборота бегемота
оборота догоняет
оборота забодает
бегемота догоняет
бегемота забодает
догоняет забодает
нагасаки атаки
нагасаки себя
нагасаки судьба
атаки себя
атаки судьба
себя судьба
товаром пожаром
товаром стрела
товаром навсегда
пожаром стрела
пожаром навсегда
стрела навсегда
упасть мать
упасть глаза
упасть себя
мать глаза
мать себя
глаза себя
фальши дальше
фальши прощай
фальши знай
дальше прощай
дальше знай
прощай знай
красная атласная
красная косу
красная

In [75]:
len(pairs0)

78456

In [34]:
from rupo.api import Engine

In [35]:
engine = Engine(language="ru")

In [37]:
engine.load('models/stress_ru_LSTM64_dropout0.2_acc99_wer8.h5', 'models/zaliznyak.txt')
engine.get_stresses("корова")

[3]

In [76]:
engine.is_rhyme("вот", "кот")

True

In [87]:
rhyme = []
for i in tqdm(range(len(pairs0))):
    p1 = pairs0[i]
    p2 = pairs1[i]          
    if engine.is_rhyme(p1, p2):
        rhyme.append(1)
    else:
        rhyme.append(0)
    if i % 10000 == 0:
        print(i, np.sum(rhyme), p1, p2)    

0 1 месте вместе
10000 614 всего восьмом
20000 1218 изменятся надеяться
30000 1827 растревожим бонч
40000 2362 простонать шило
50000 2933 провод батарее
60000 3555 числами искренние
70000 4128 скоро ночам
80000 4747 трусцой чётче
90000 5326 круто ебанутый


In [88]:
np.sum(rhyme)

5557

In [89]:
for i in tqdm(range(len(pairs0))):
    p1 = pairs0[i]
    p2 = pairs1[i]
    
    if p1[-3:]==p2[-3:] and rhyme[i]==0 :
        rhyme[i] = 2
        print(p1, p2)

#     if i % 10000 == 0:
#         print(i, np.sum(rhyme), p1, p2)    

жить бомбить
нагасаки атаки
красная атласная
стоном тоном
глазами нами
глазами нами
любить повторить
встречены заколочены
агенты ингридиенты
скрылось остановилось
части масти
шалости радости
шалости усталости
шалости старости
радости усталости
радости старости
усталости старости
ссужился мужился
получается надвигаются
звонки позвонки
вспять менять
фараоны вагоны
зала продавала
формы платформы
вызов релизов
ёрков осколков
мгновение дуновение
придется вернутся
плавки удавки
качают посещают
сигарета света
поднять менять
ускорялся разбивался
застроенным достойным
застроенным достойным
дороже тоже
сцене фене
смущается нравится
пела кипела
тосты кусты
навеки реки
бумаги универмаги
прости глупости
остыла пыла
монолит болит
живой твой
глаза раза
полушепотом животом
торопись вернись
красиво пиво
путь свернуть
достали попали
пацаны штаны
дорогу ногу
помешать бросать
вдаль жаль
перо пьеро
ангоры разговоры
культуры прокуратуры
приставали знали
психорифмапаркур накур
вдаль печаль
любопытства естест

петровна константиновна
глаза раза
метро утро
марш-бросок кусок
марш-бросок сок
кусок сок
стены вены
дранными первыми
побеждать рвать
цивильный ювелирный
понятно приятно
грубости сущности
плате плакате
потом шопотом
потом опытом
шопотом опытом
вами благами
началось лось
каски соски
досуга друга
мечтал встал
произвольно больно
кнут мокнут
теме системе
бетона зона
совесть невесомость
никогда всегда
размер пример
говно равно
иванов штанов
смалось осталось
власть часть
класть часть
рэп полурэп
рэп полурэп
пожара ягуара
мало ебало
приготовь кровь
света рикошета
цель мадемуазель
сеньорите внемлите
сеньорите увидите
внемлите увидите
вправо браво
хочешь скажешь
эверест мест
эверест надоест
эверест бест
мест надоест
надоест бест
жиже париже
жиже поближе
хитро интро
деле еле-еле
конфетка малолетка
осторожен возможен
жить любить
репертуаре баре
жить любить
останется тянется
сражаются получается
сурово слово
грядущий жгущий
идет будет
кипело дело
важней дней
толку иголку
батут тут
сможешь поможешь

кругозор дозор
участи прости
лежим турборежим
храма мама
песком босиком
мечта мачта
спишь молчишь
сомнения положения
диктатуры фигуры
повторяются оступится
повторяются смирится
исправляются оступится
исправляются смирится
оступится смирится
строчки точки
субботу пилоту
сказалось показалось
естрелла изабелла
естрелла моцарелла
гаражами сами
корешами сами
патриарших наших
вопросов утёсов
собой бой
осколки иголки
родным свободным
нами руками
портмоне моне
собой бой
жесть совесть
джунглей клей
шестой той
развлечиться обжечься
ясно опасно
поверь дверь
сколько только
тоже похоже
одновременно неизменно
чувства искусства
заколочу заточу
метро нутро
спотыкаюсь маюсь
остается бьется
остановился скрылся
палево порево
звери потери
цвета брослета
цвета трандулета
брослета трандулета
деда велосипеда
кошка окошка
буду вуду
ночь помочь
девчонок кинопленок
дрались остались
бравада ада
снится пригодится
русь вернусь
чмары гитары
чмары гитары
кровь любовь
сердится получится
слышишь тишь
встречный конкрет

конкретный стопроцентный
ценится целится
парень печень
фонарей добрей
верится поместится
взлетаем таем
повода холода
давались валялись
брали печали
понять погулять
переиграет понимает
давайте переживайте
счёта почёта
упор спор
меняется ломаются
получается объединяются
получается сливаются
объединяются сливаются
путь свернуть
провода вода
рядом трудом
подстать сказать
подстать сказать
подстать наплевать
сказать наплевать
сказать наплевать
капкана плана
капитала зала
теперь дверь
рад тирад
зад назад
качают величают
знаешь получаешь
боже дороже
чёрно-белый белый
знакомы комы
путь вернуть
веки чеки
цены сцены
шах ушах
власти масти
прочь ночь
опять взять
опять взять
прочь дочь
прочь ночь
однобоко око
врубаюсь парюсь
врубаюсь парюсь
врубаюсь парюсь
вопросы носы
вопросы взносы
носы взносы
вина вечерина
говоришь видишь
братва битва
забытый сытый
трансе альянсе
ходить остановить
ходить говорить
ходить остановить
остановить говорить
говорить остановить
вновь любовь
нами местами
вернуть путь
стар

пела смотрела
ленточкой веточкой
ярко жарко
принцем сердцем
колыбели горели
реплик блик
высекал зеркал
ловите вите
сильно стабильно
рассчитаны пропитаны
наживы перспективы
голосом вопросом
руки внуки
маты зарплаты
синяк отходняк
гнули пули
перемешалось осталось
жалость ярость
дежурно дурно
жала наливала
тусонула продула
обещание здание
стараний знаний
потели терпели
подглядывать подслушивать
слова снова
шепотом потом
драть делать
again rain
вести обрести
жить изменить
одиноко око
загораясь заполняясь
чёрный учёный
пиздят сидят
конфликтовать плавать
поэтом летом
приблизиться заколоситься
молодость малость
молодость малость
алчность бездарность
коварности жадности
кретины глины
кретины зины
глины зины
глины бобины
зины бобины
сложно осторожно
трёшка картошка
измена члена
смотри изнутри
спросом носом
джемы проблемы
дорога бога
тру ветру
узорное беспризорное
тополиным длинным
собирались дрались
вата автомата
стали кричали
угодно модно
зажигается взрывается
тактичность личность
котлах жилах

In [90]:
np.sum(rhyme)

13387

In [92]:
for i in tqdm(range(len(pairs0))):
    p1 = pairs0[i]
    p2 = pairs1[i]
    
    if len(p1) >= 3:
        for j in range(len(p1) - 3):
            trigram = p1[j:j+3]
            if p2.count(trigram)==1 and rhyme[i]==0 :
                rhyme[i] = 3
                print(p1, p2)

свете ветер
ритм говорит
паутину тине
вечер замечено
растворителем родителям
вкусом мусор
случаю лучшее
верила дверью
вагонами регионах
трахает мономаха
помогут богу
симпатии пати
атлас шатлов
набережной нежно
шоколадом надо
вечер свечи
константа места
места известно
милениума заведений
косметике поэтику
неприступна крупно
продаётся года
толпы толк
дорогу стороны
стоит устои
стоит пустое
станет титаник
растения застенок
наркозависим завтрак
наколке волки
двери вернешь
осталось стало
быстрей острее
постели постелем
постели стены
постелем стены
курнуть урны
черный урны
домом домам
заразу разом
склифе клипов
истину стимул
сожаленья поколеньем
простынь противень
болото волокна
девяносто восток
просто восток
сэкса экстрасенсов
прежде надежды
похожим рожи
место чести
теста чести
всевышний вышли
встали оставался
голос волосы
вода небосвод
приятелем наблюдатель
настрой страз
цветах ветров
вваливали проталинам
макухами кухне
снова нового
сабвуфер фуфел
оставаться двенадцать
постели телик
постел

чисто альтависта
головоломки человек
сторонам слона
классе власть
волшебство ствол
прополоскать тисках
прополоскать сканк
прополоскать носках
тисках сканк
сканк носках
городская раскат
городская выпускать
раскат выпускать
пускай отпуская
пускай скат
отпуская каскад
отпуская скат
каскад скат
пленительны закалены
переговоры вперёд
информации нация
безработица заботятся
мерзко дерзкими
страсти счастья
сам-сам-самнениях исключения
гения настроение
гения настроение
подростка тосты
подростка просьбе
подростка гости
тосты гости
поднял западня
осталось радость
станций иностранца
разному браза
тороплюсь плюс
пидором идолы
сокровенном мгновенье
накрошу хорошо
лекарство царствах
плеер жалеешь
веры вверх
высоток фотоальбоме
высоток фотоальбоме
осколки полках
теченьем меньше
цени цене
траблом праблэм
совсем всех
драма килограммов
асфальте машинально
машинально журнале
знаки узнаем
арестом местных
композитором битой
меня перемене
вопросы просто
ботинки никотин
смейтесь пейте
причины капучино
лето би

вместе мести
мостов стоишь
мостов
дурно урна
повесил весело
телевизора видимо-невидимо
съеденными неделями
автора завтра
ругались скандалист
разу зараза
заплакал плачет
раскладом надо
очевидцев видели
года году
ластиком одноклассников
видео возненавидела
любовью любовь
любовью любовь
читать читаешь
просим прошло
видеть обидел
планом рано
ноги многому
чувство присутствие
просто вопроса
глупо лупой
марки иномарках
иномарках подарки
коротком сковородке
запятых распятье
сони зафинтесонить
зафинтесонить брони
зафинтесонить гонит
по-разному безобразия
потоп потом
дома чередом
счастье части
умереть смерть
стихотворениям одупление
тяжко затяжка
отвечаю меча
биточек почерк
биточек почерк
пойму пойми
занавес сыграна
подняться поменять
вереницей столице
вереницей столице
рекламы треканов
позитивной спортивном
климатгорылюдикраныбашни вчерашний
горд город
хит-парадах мкада
былью крыльями
всевышний лишних
вместе честен
жестоко столы
брендом конкурентом
гастролей суперпупермегастар
одного одни
телом

слабостей гадости
яркая доярками
счастье растопляя
ноги многие
параллельное паразиты
русаков закон
переговоров мозгов
историй стоит
лекарств государство
просто вопросов
заготовок повод
однажды граждан
ошибкой шибко
незасвеченных незамеченной
плечи лечит
растёт зарастет
завесили месива
сроком строки
длинные линий
человеком весело
пороге строгим
неадеквате хватит
усталость старт
осталось старт
грустно пусто
вечеринке пластинки
центрам проценты
остаться старше
переменах время
система стены
потратить кровати
ростова невесомость
ростова невесомость
ростова невесомость
ха-ай а-ау-а
дава-а-ай а-ау-а
просто запапироси-и-им
выносит запапироси-и-им
друга округе
части счастье
отметим дети
подпевке девкам
наброски просто
подскажешь покажи
личный отличий
карты партией
партой партией
напряжение женя
дружба оружием
прежним бережно
прежним нежность
бережно нежность
пользу пользуется
стерва стервятник
предательства обязательно
бродам кислородом
соответственно бедствие
совместно местности
ресурсы курсе


In [93]:
np.sum(rhyme)

22963

In [94]:
d = {'word1':pairs0, 'word2': pairs1, 'rhyme':rhyme}
df = pd.DataFrame(d)
df  = df.drop_duplicates()
df.shape

(89065, 3)

In [95]:
df  = df.drop_duplicates()
df.shape

(89065, 3)

In [96]:
df.head(200)

,word1,word2,rhyme
0,месте,вместе,1
1,месте,тему,0
2,месте,смело,0
3,вместе,тему,0
4,вместе,смело,0
5,тему,смело,0
6,сердцах,руках,0
7,сердцах,вру,0
8,сердцах,дышу,0
9,руках,вру,0


In [97]:
df.shape

(89065, 3)

In [98]:
df.to_csv('data/rhymes.csv', sep=';', index=False)